# Comfy Kettenregel (Autograd DIY) - univariate, skalare Funktionen

$$F(x) = f_1 \circ f_2 = f_1(f_2(x)) \Rightarrow f_1'(f_2(x)) \cdot f'_2(x)$$

$$F(x) = f_1 \circ f_2 \circ f_3 = f_1(f_2(f_3(x))) \Rightarrow f_1'(f_2(f_3(x))) \cdot f_2'(f_3
(x)) \cdot f_3'(x)$$

## Aufgabe

Ziel: Gradientenbasierte Optimierung von $f(x) = \sqrt{\frac{1}{e^{\sin(x)}}}$


### 0. Imports

In [1]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### 1.0 Operationen definieren

In [7]:
def one_div_x(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = 1 / x
    derivative = -inner_derivative / x**2

    return value, derivative


def sin(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.sin(x)
    derivative = math.cos(x) * inner_derivative

    return value, derivative


def sqrt(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.sqrt(x)
    derivative = 1 / (2 * math.sqrt(x)) * inner_derivative

    return value, derivative


def exp(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.exp(x)
    derivative = math.exp(x) * inner_derivative

    return value, derivative

### 1.1 Funktionsdefinition

In [ ]:
def f_x(x: float) -> tuple[float, float]:

    return sqrt(*one_div_x(*exp(*sin(x))))

### 2. Gradient Descent

In [4]:
x_start = 4.0  # starting value
x_min = x_start - 8.0  # x-axis limits
x_max = x_start + 8.0
xs = []  # values for the animation
ys = []

lr = 1e-2  # step size
significant_gradient = 1e-3  # termination criteria
iter = 1  # counter

while True:
    y, deriv = f_x(x_start)
    if np.fabs(deriv) >= significant_gradient:
        xs.append(x_start)
        ys.append(y)
        x_start -= lr * deriv
        print(iter, x_start, y) if iter % 100 == 0 or iter == 1 else None
    else:
        xs.append(x_start)
        ys.append(y)
        break
    iter += 1

1 3.9952285694680634 1.4599486263271053
100 3.4637020821138367 1.1745896956823383
200 2.954068550509694 0.9130108589989183
300 2.57138696808758 0.7644917145666532
400 2.298568010801605 0.6889654489398163
500 2.103204330028916 0.6502665163644317
600 1.9617690480759373 0.629997349667761
700 1.8585492144744415 0.6192063048870152
800 1.7828476651424012 0.6134034622298884
900 1.7271709198977527 0.6102649804942161
1000 1.6861571631419414 0.608562044769035
1100 1.6559183672370383 0.6076363876214959
1200 1.6336130947293028 0.6071327404432841
1300 1.61715557525729 0.606858561103853
1400 1.6050109715735437 0.6067092576902453
1500 1.5960483265235055 0.6066279420955654
1600 1.589433666685135 0.6065836510829978
1700 1.5845517664862814 0.6065595254954701
1800 1.5809486710850136 0.6065463838009004
1900 1.5782893816209955 0.6065392251541556
2000 1.5763266672980671 0.606535325610263
2100 1.5748780640419968 0.606533201395113


### 3.0 Funktionsplot

In [5]:
x = np.arange(x_min, x_max, 0.01)

res = [f_x(_) for _ in x]
y, derivative = zip(*res)

df = pd.DataFrame(
    {
        "x": x,
        "y": y,
        "derivative": derivative,
    }
)

px.line(df, x="x", y="y")

### 3.1 Animation

In [6]:
# get the values
x = np.arange(x_min, x_max, 0.01)

res = [f_x(_) for _ in x]
y, _ = zip(*res)

# define both graphs
fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            line=dict(color="green", width=1),
            name="Function Graph",
        ),
        go.Scatter(
            x=[xs[0]],
            y=[ys[0]],
            mode="markers",
            marker=dict(color="red", size=10),
            name="Current Position",
        ),
    ]
)

# update layout parameters and add start button for animation
fig.update_layout(
    width=1400,
    height=900,
    xaxis=dict(range=(x_min, x_max), autorange=False),
    yaxis=dict(range=(np.min(y) - 0.5, np.max(y) + 0.5), autorange=False),
    title_text="Gradient Descent Animation",
    # start button config
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 5, "redraw": False},
                            "fromcurrent": True,
                            "transition": {"duration": 0, "easing": "linear"},
                        },
                    ],
                    label="start",
                    method="animate",
                )
            ],
        )
    ],
)

# specify the animation frames
fig.update(
    frames=[
        go.Frame(data=[go.Scatter(x=[xs[k]], y=[ys[k]])], traces=[1])
        for k in range(len(ys))
    ]
)

# show result
fig.show()

# 2024-11-18

Bisherige Ansatz hat folgende Limitierungen
- funktioniert nur für Ausdrücke in geschlossener Form, keine Kontrollflusslogik
- inkompatibel mit binären Operatoren (+, *, ...)
- funktioniert nur in 1D

In [ ]:
def f(a, b, c) -> float:
    # (((b**2) * c) + a)
    x = b**2 * c
    y = a + x
    return y



In [ ]:
from __future__ import annotations


class Value:
    def __init__(self, value: float, ancestors: tuple[Value, ...] = (), name=""):
        self.value = value
        self.ancestors = ancestors
        self.name = name

    def __add__(self, other: Value) -> Value:
        result = Value(self.value + other.value, (self, other))
        return result

    def __sub__(self, other: Value) -> Value:
        pass

    def __mul__(self, other: Value) -> Value:
        pass

    # TODO: Floatingpointdivision
    # TODO: Potenzierung (x**n)
    # TODO: Negation
    # TODO: Vergleichsoperatoren ==, >=, <=, <, >

    def build_graph(self) -> dict:
        # Vorschlag einer Repräsentation des DAG (directed, acyclical graph)
        # graph = {
        #     "y": [
        #         {"a": []},
        #         {
        #             "x": [
        #                 {
        #                     "c": [],
        #                 },
        #                 {
        #                     "": [
        #                         {"b": []},
        #                     ]
        #                 },
        #             ]
        #         },
        #     ]
        # }
        pass

    @staticmethod
    def plot_graph(graph_dict: dict):
        # "graph visualization python", graphviz
        pass

x = Value(5, name="x")
y = Value(2.5, name="y")

# TODO: Folgendes sollte ausfühbar sein:

# x + y
# x * y
# x - y
# x / y
# -x

# def foo(a: Value, b: Value, c: Value):
#     if a > 2:
#         a * b + c
#     return a - b * c

# a = Value(2.5)
# b = Value(3)
# c = Value(1)

# z1 = foo(a, b, c)
# graph = z1.build_graph()
# Value.plot_graph(graph)

# z2 = foo(Value(-1), b, c)
# graph = z2.build_graph()
# Value.plot_graph(graph)


TypeError: unsupported operand type(s) for +: 'Value' and 'Value'

In [ ]:
graph = {
    "y": [
        {"a": []},
        {
            "x": [
                {
                    "c": [],
                },
                {
                    "": [
                        {"b": []},
                    ]
                },
            ]
        },
    ]
}